In [56]:
#importing the required libraries and packages
import pandas_datareader as web
import pandas as pd
import numpy as np
import datetime as dt
import math
import warnings

warnings.filterwarnings("ignore")


In [72]:
#extracting the data 
stocks=[]
f = open("symbols1.txt","r")
for line in f:
  stocks.append(line.strip())
f.close()
web.DataReader(stocks,"yahoo",start="2000-1-1",end="2019-12-31")["Close"].to_csv("price.csv")
web.DataReader(stocks,"yahoo",start="2000-1-1",end="2019-12-31")["Volume"].to_csv("Volume.csv")

In [74]:
#saving the data inhe variable
prices = pd.read_csv("price.csv", index_col="Date", parse_dates=True)
volumechanges = pd.read_csv("Volume.csv", index_col="Date", parse_dates=True).pct_change()*100


In [76]:
#initialising the global variables
today = dt.date(2000, 1, 15)
simend = dt.date(2019, 12, 31)
tickers = []
transactionid = 0
money = 1000000
portfolio = {}
activelog = []
transactionlog = []


In [77]:
#function to exctract the current stock price
def getprice(date, ticker):
    global prices
    return prices.loc[date][ticker]


In [78]:
#transanction function
def transaction(id, ticker, amount, price, type, info):
    global transactionid
    if type == "buy":
        exp_date = today + dt.timedelta(days=14)
        transactionid += 1
    else:
        exp_date = today
    if type == "sell":
        data = {"id": id, "ticker": ticker, "amount": amount, "price": price, "date": today, "type": type,
                "exp_date": exp_date, "info": info}
    elif type == "buy":
        data = {"id": transactionid, "ticker": ticker, "amount": amount, "price": price, "date": today, "type": type,
                "exp_date": exp_date, "info": info}
        activelog.append(data)
    transactionlog.append(data)

In [79]:
#function for stock buy
def buy(interestlst, allocated_money):
    global money, portfolio
    for item in interestlst:
        price = getprice(today, item)
        if not np.isnan(price):
            quantity = math.floor(allocated_money/price)
            money -= quantity*price
            portfolio[item] += quantity
            transaction(0, item, quantity, price, "buy", "")

In [80]:
#function for stock sell
def sell():
    global money, portfolio, prices, today
    itemstoremove = []
    for i in range(len(activelog)):
        log = activelog[i]
        if log["exp_date"] <= today and log["type"] == "buy":
            tickprice = getprice(today, log["ticker"])
            if not np.isnan(tickprice):
                money += log["amount"]*tickprice
                portfolio[log["ticker"]] -= log["amount"]
                transaction(log["id"], log["ticker"], log["amount"], tickprice, "sell", log["info"])
                itemstoremove.append(i)
            else:
                log["exp_date"] += dt.timedelta(days=1)
    itemstoremove.reverse()
    for elem in itemstoremove:
        activelog.remove(activelog[elem])

In [81]:
#simulation function
def simulation():
    global today, volumechanges, money
    start_date = today - dt.timedelta(days=14)
    series = volumechanges.loc[start_date:today].mean()
    interestlst = series[series > 100].index.tolist()
    sell()
    if len(interestlst) > 0:
        #moneyToAllocate = 500000/len(interestlst)
        moneyToAllocate = currentvalue()/(2*len(interestlst))
        buy(interestlst, moneyToAllocate)

In [82]:
#helping function
def getindices():
    global tickers
    f = open("symbols1.txt", "r")
    for line in f:
        tickers.append(line.strip())
    f.close()

In [83]:
def tradingday():
    global prices, today
    return np.datetime64(today) in list(prices.index.values)


In [84]:
def currentvalue():
    global money, portfolio, today, prices
    #print(money)
    value = money
    for ticker in tickers:
        tickprice = getprice(today, ticker)
        if not np.isnan(tickprice):
            value += portfolio[ticker]*tickprice
    return int(value*100)/100

In [86]:
#main body of the simulation
def main():
    global today
    getindices()
    for ticker in tickers:
        portfolio[ticker] = 0
    while today < simend:
        while not tradingday():
            today += dt.timedelta(days=1)
        simulation()
        currentpvalue = currentvalue()
        #yesterday=today
        today += dt.timedelta(days=7)
    print(currentpvalue, today)
main()

1147668.2 2019-12-31
